In [1]:
# Pascal VOC ( Visual Object Class )
# 20 Classes outdated for present scenerio
# You Only Look Once : Unified Real Time Object Detection


In [1]:
import torch
import torch.nn as nn
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import os
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as FT
from tqdm import tqdm
from torch.utils.data import DataLoader

seed = 123
torch.manual_seed(seed)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Download the dataset
!kaggle datasets download -d aladdinpersson/pascalvoc-yolo

Dataset URL: https://www.kaggle.com/datasets/aladdinpersson/pascalvoc-yolo
License(s): unknown
100% 4.29G/4.31G [00:27<00:00, 151MB/s]
100% 4.31G/4.31G [00:27<00:00, 168MB/s]


In [4]:
!unzip -q /content/pascalvoc-yolo.zip

In [5]:


architecture_config = [
    (7, 64, 2, 3), # [ kernel_size, number_of_filters or output_channels, stride, padding]
    "M", # Maxpool
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    #List
    # [ conv1, conv2, number of times this pattern is repeated ] similar to Inception (bottleneck layer used)
    [(1,256, 1, 0), (3, 512, 1, 1), 4],
     (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1),
]

In [6]:
class CNNBlock( nn.Module):

  def __init__(self, in_channels, out_channels, **kwargs):
    super(CNNBlock, self).__init__() #initialization of parent class

    self.conv = nn.Conv2d( in_channels, out_channels, bias = False, **kwargs )
    self.batchnorm = nn.BatchNorm2d( out_channels )
    self.leakyrelu = nn.LeakyReLU(0.1)

  def forward(self, x):
    return self.leakyrelu(self.batchnorm(self.conv(x)))




In [7]:
class YoloV1(nn.Module):
  def __init__(self, in_channels = 3 , **kwargs):
    super(YoloV1, self).__init__()
    self.architecture = architecture_config
    self.in_channels = in_channels
    self.darknet = self._create_conv_layers(self.architecture)
    # Conv layer in YOLO . This Darknet can be replaced with Pre Trained ResNet State of Art for better training and result and resize the input size of image
    self.fc = self._create_fc(**kwargs)

  def forward(self, x):
    x = self.darknet(x)
    # print(x.shape)
    return self.fc(torch.flatten(x, start_dim =1)) # start_dim =1 because we dont want to flatten example/batch_size

  def _create_conv_layers(self, architecture):
    layers = []
    in_channels = self.in_channels

    for x in architecture:
      if type(x) ==tuple:
        layers += [
            CNNBlock( in_channels, x[1], kernel_size = x[0], stride = x[2] , padding = x[3])
        ]
        in_channels = x[1]
      elif type(x) ==str:
        layers +=[ nn.MaxPool2d( kernel_size =(2,2), stride =(2,2) )]

      elif type(x) == list:
        conv1 = x[0]
        conv2 = x[1]
        num_repeats = x[2]

        for _ in range(num_repeats):
          layers += [
              CNNBlock( in_channels, conv1[1], kernel_size = conv1[0], stride = conv1[2], padding = conv1[3])
          ]

          layers += [
              CNNBlock( conv1[1], conv2[1], kernel_size = conv2[0], stride = conv2[2], padding = conv2[3])
          ]

          in_channels = conv2[1]

    return nn.Sequential(*layers)

  def _create_fc(self, split_size, num_boxes, num_classes):
    S, B, C = split_size, num_boxes, num_classes
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear( 1024 * S * S, 496),
        nn.Dropout(0.0),
        nn.LeakyReLU(0.1),
        nn.Linear(496, S * S * (C + B * 5)) # (S,S, 30 to  C+B+S)
    )


In [8]:
# Predicting 2 bounding box


In [9]:
def tesst(S =7, B = 2, C = 20):
  model = YoloV1(split_size=S, num_boxes=B, num_classes=C)
  x= torch.randn((2, 3, 448, 448))
  print(model(x).shape)

In [10]:
tesst()

torch.Size([2, 1470])


In [11]:
# intersetion over union
# non max suppression
# mean average precision
# cellboxes to boxes
# get_bounding boxes
# plot image
# save checkpoint
# load checkpoint

In [12]:
def intersection_over_union( boxes_preds, boxes_labels, box_format = 'midpoint'):

  if box_format == "midpoint":
    box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
    box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
    box1_x2 = boxes_preds[..., 2:3] + boxes_preds[..., 2:3] / 2
    box1_y2 = boxes_preds[..., 3:4] + boxes_preds[..., 3:4] / 2
    box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
    box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
    box2_x2 = boxes_labels[..., 2:3] + boxes_labels[..., 2:3] / 2
    box2_y2 = boxes_labels[..., 3:4] + boxes_labels[..., 3:4] / 2

  if box_format == "corners":
    box1_x1 = boxes_preds[..., 0:1] # to keep the dimension we use slicing
    box1_y1 = boxes_preds[..., 1:2]
    box1_x2 = boxes_preds[..., 2:3]
    box1_y2 = boxes_preds[..., 3:4]
    box2_x1 = boxes_labels[..., 0:1]
    box2_y1 = boxes_labels[..., 1:2]
    box2_x2 = boxes_labels[..., 2:3]
    box2_y2 = boxes_labels[..., 3:4]

  x1 = torch.max( box1_x1, box2_x1)
  x2 = torch.max( box1_x2, box2_x2)
  y1 = torch.min( box1_y1, box2_y1)
  y2 = torch.min( box1_y2, box2_y2)

  intersection = ( x2 - x1 ).clamp(0) * ( y2 - y1 ).clamp(0) # covering edge case of no intersection

  box1_area = abs( (box1_x2 - box1_x1) * (box1_y2 - box1_y1))
  box2_area = abs( (box2_x2 - box2_x1) * (box2_y2 - box2_y1))

  return intersection / (box1_area + box2_area - intersection + 1e-6)



In [13]:
def nms( bboxes, iou_threshold, threshold, box_format = 'corners'):
  """
  Does Non Max Suppression given bboxes

  Parameters:
    bboxes (list) : list of lists containing all bboxes with each bboxes
    specified as [class_pred, prob_score, x1, y1, x2, y2]
    iou_threshold (float) : threshold where predicted bboxes is correct
    threshold (float) : threshold to remove predicted bboxes ( independent of IoU )
    box_format (str) : "midpoint" or "corners" used to specify bboxes

  Returns :
    list : bboxes after performing NMS given a specific IoU threshold
  """

  assert type(bboxes) ==list

  bboxes = [ box for box in bboxes if box[1] > threshold]
  bboxes = sorted(bboxes, key = lambda x: x[1], reverse= True )
  bboxes_after_nms = []

  while bboxes:
    chosen_box = bboxes.pop(0)
    bboxes = [
        box for box in bboxes
        if box[0] != chosen_box[0]
        or intersection_over_union(
            torch.tensor(chosen_box[2:]),
            torch.tensor(box[2:]),
            box_format = box_format
        )
        < iou_threshold
    ]

    bboxes_after_nms.append(chosen_box)

  return bboxes_after_nms



In [14]:
def mean_average_precision( pred_boxes, true_boxes, iou_threshold = 0.5, box_format = 'corners', num_classes = 20):
  """
  Calculates mean average precision

  Parameters:
    pred_boxes (list) : list of lists containing all bboxes with each bboxes
    specified as [train_idx, class_prediction, prob_score, x1, y1, x2, y2]
    true_boxes (list) : same as pred_boxes except all the correct ones
    iou_threshold (float) : threshold where predicted bboxes is correct
    box_format (str) : "midpoint" or "corners" used to specify bboxes
    num_classes (int) : number of classes

  Returns:
    float : mAP value across all classes given a specific IoU threshold
  """

  # list storing all the AP for respective classes
  average_precisions = []

  # used for numerical stability later on
  epsilon = 1e-6

  for c in range(num_classes):
    detections = []
    ground_truths = []

    # go through all predictions and targets
    # and only the ones that belong to the current class c
    for detection in pred_boxes:
      if detection[1] == c:
        detections.append(detection)

    for true_box in true_boxes:
      if true_box[1] == c:
        ground_truths.append(true_box)

    # find the amount of bboxes for each training example
    # Counter here finds how many ground truth bboxes we get
    # for each training example, so let's say img 0 has 3,
    # img 1 has 5 then we will obtain a dictionary with:
    # amount_bboxes = {0:3, 1:5}
    amount_bboxes = Counter([gt[0] for gt in ground_truths])

    # We then go through each key, val in this dictionary
    # and convert to the following (w.r.t. same example):
    # amount_bboxes = { 0:torch.tensor[0,0,0], 1:torch.tensor[0,0,0,0,0]}
    for key, val in amount_bboxes.items():
      amount_bboxes[key] = torch.zeros(val)

    # sort by box probabilities which is index 2
    detections.sort(key = lambda x : x[2] , reverse = True)
    TP = torch.zeros((len(detections)))
    FP = torch.zeros((len(detections)))
    total_true_bboxes = len(ground_truths)

    # if none exists for thsi class then we can safely skip
    if total_true_bboxes == 0:
      continue

    for detection_idx, detection in enumerate(detections):
      ground_truth_img = [bbox for bbox in ground_truths if bbox[0] == detection[0]]

      num_gts = len(ground_truth_img)
      best_iou = 0

      for idx, gt in enumerate(ground_truth_img):
        iou = intersection_over_union(
            torch.tensor(detection[3:]),
            torch.tensor(gt[3:]),
            box_format = box_format,
        )

        if iou > best_iou:
          best_iou = iou
          best_gt_idx = idx

      if best_iou > iou:
        # only detect ground truth detection once
        if amount_bboxes[detection[0]][best_gt_idx] == 0:
          # true positive and add this bounding box to seen
          TP[detection_idx] = 1
          amount_bboxes[detection[0]][best_gt_idx] =1

        else:
          FP[detection_idx] = 1

      # if IOU is lower then the detection is a false positive
      else:
        FP[detection_idx] = 1

    TP_cumsum = torch.cumsum(TP, dim = 0)
    FP_cumsum = torch.cumsum(FP, dim = 0)
    recalls = TP_cumsum / (total_true_bboxes + epsilon)
    precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
    precisions = torch.cat((torch.tensor([1]), precisions))
    recalls = torch.cat((torch.tensor([0]), recalls))
    # torch.trapz for numerical integrationq
    average_precisions.append(torch.trapz(precisions, recalls))

  return sum(average_precisions) / len(average_precisions)


In [15]:
class TotalLoss( nn.Module):

  def __init__(self, S = 7, B = 2, C = 20):
    super(TotalLoss, self).__init__()

    self.mse = nn.MSELoss(reduction = "sum")
    self.S = S
    self.B = B
    self.C = C
    self.lambda_no_obj = 0.5
    self.lambda_coord = 5

  def forward( self, predictions, target):
    print("Dewesh")
    print(predictions.shape)
    predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)
    print(predictions.shape)
    iou_b1 = intersection_over_union( predictions[..., 21:25], target[..., 21:25])
    iou_b2 = intersection_over_union( predictions[..., 26:30], target[..., 21:25])
    ious = torch.cat( [iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim =0)
    iou_maxes, best_box = torch.max(ious, dim = 0)
    exists_box = target[..., 20].unsqueeze(3) # identity of obj_i

    # For Box Coordinates
    box_predictions = exists_box * (
      (  best_box * predictions[..., 21:25] + (1 - best_box) * predictions[..., 26:30] )
    )
    box_targets = exists_box * target[..., 21:25]

    box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4]) * torch.sqrt(
        torch.abs(box_predictions[..., 2:4] + 1e-6)
    )

    box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])

    box_loss = self.mse(
        torch.flatten(box_predictions, end_dim = -2),
        torch.flatten(box_targets, end_dim =-2)
    )


    # For Object Loss
    pred_box = (
        best_box * predictions[..., 25:26] + ( 1- best_box) * predictions[..., 20:21]
    )

    object_loss = self.mse(
        torch.flatten(exists_box * pred_box),
        torch.flatten(exists_box * target[..., 20:21])
    )

    # For no object loss

    no_obj_loss = self.mse(
        torch.flatten( ( 1 - exists_box) * predictions[..., 20:21], start_dim = 1),
        torch.flatten( ( 1- exists_box ) * target[..., 20:21], start_dim = 1)
    )

    no_obj_loss += self.mse(
        torch.flatten( ( 1 - exists_box) * predictions[..., 25:26], start_dim = 1),
        torch.flatten( ( 1- exists_box ) * target[..., 20:21], start_dim = 1)
    )

    # class Loss
    class_loss = self.mse(
        torch.flatten(exists_box * predictions[..., :20], end_dim = -2),
        torch.flatten(exists_box * target[..., :20], end_dim = -2)

    )

    # total loss
    loss = (
        self.lambda_coord * box_loss
        + object_loss
        + self.lambda_no_obj * no_obj_loss
        + class_loss
    )

    return loss



In [23]:

def plot_image(image, boxes):
    """Plots predicted bounding boxes on the image"""
    im = np.array(image)
    height, width, _ = im.shape

    # Create figure and axes
    fig, ax = plt.subplots(1)
    # Display the image
    ax.imshow(im)

    # box[0] is x midpoint, box[2] is width
    # box[1] is y midpoint, box[3] is height

    # Create a Rectangle potch
    for box in boxes:
        box = box[2:]
        assert len(box) == 4, "Got more values than in x, y, w, h, in a box!"
        upper_left_x = box[0] - box[2] / 2
        upper_left_y = box[1] - box[3] / 2
        rect = patches.Rectangle(
            (upper_left_x * width, upper_left_y * height),
            box[2] * width,
            box[3] * height,
            linewidth=1,
            edgecolor="r",
            facecolor="none",
        )
        # Add the patch to the Axes
        ax.add_patch(rect)

    plt.show()

def get_bboxes(
    loader,
    model,
    iou_threshold,
    threshold,
    pred_format="cells",
    box_format="midpoint",
    device="cpu",
):
    all_pred_boxes = []
    all_true_boxes = []

    # make sure model is in eval before get bboxes
    model.eval()
    train_idx = 0

    for batch_idx, (x, labels) in enumerate(loader):
        x = x.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            predictions = model(x)

        batch_size = x.shape[0]
        true_bboxes = cellboxes_to_boxes(labels)
        bboxes = cellboxes_to_boxes(predictions)

        for idx in range(batch_size):
            nms_boxes = nms(
                bboxes[idx],
                iou_threshold=iou_threshold,
                threshold=threshold,
                box_format=box_format,
            )


            #if batch_idx == 0 and idx == 0:
            #    plot_image(x[idx].permute(1,2,0).to("cpu"), nms_boxes)
            #    print(nms_boxes)

            for nms_box in nms_boxes:
                all_pred_boxes.append([train_idx] + nms_box)

            for box in true_bboxes[idx]:
                # many will get converted to 0 pred
                if box[1] > threshold:
                    all_true_boxes.append([train_idx] + box)

            train_idx += 1

    model.train()
    return all_pred_boxes, all_true_boxes



def convert_cellboxes(predictions, S=7):
    """
    Converts bounding boxes output from Yolo with
    an image split size of S into entire image ratios
    rather than relative to cell ratios. Tried to do this
    vectorized, but this resulted in quite difficult to read
    code... Use as a black box? Or implement a more intuitive,
    using 2 for loops iterating range(S) and convert them one
    by one, resulting in a slower but more readable implementation.
    """

    predictions = predictions.to("cpu")
    batch_size = predictions.shape[0]
    predictions = predictions.reshape(batch_size, 7, 7, 30)
    bboxes1 = predictions[..., 21:25]
    bboxes2 = predictions[..., 26:30]
    scores = torch.cat(
        (predictions[..., 20].unsqueeze(0), predictions[..., 25].unsqueeze(0)), dim=0
    )
    best_box = scores.argmax(0).unsqueeze(-1)
    best_boxes = bboxes1 * (1 - best_box) + best_box * bboxes2
    cell_indices = torch.arange(7).repeat(batch_size, 7, 1).unsqueeze(-1)
    x = 1 / S * (best_boxes[..., :1] + cell_indices)
    y = 1 / S * (best_boxes[..., 1:2] + cell_indices.permute(0, 2, 1, 3))
    w_y = 1 / S * best_boxes[..., 2:4]
    converted_bboxes = torch.cat((x, y, w_y), dim=-1)
    predicted_class = predictions[..., :20].argmax(-1).unsqueeze(-1)
    best_confidence = torch.max(predictions[..., 20], predictions[..., 25]).unsqueeze(
        -1
    )
    converted_preds = torch.cat(
        (predicted_class, best_confidence, converted_bboxes), dim=-1
    )

    return converted_preds


def cellboxes_to_boxes(out, S=7):
    converted_pred = convert_cellboxes(out).reshape(out.shape[0], S * S, -1)
    converted_pred[..., 0] = converted_pred[..., 0].long()
    all_bboxes = []

    for ex_idx in range(out.shape[0]):
        bboxes = []

        for bbox_idx in range(S * S):
            bboxes.append([x.item() for x in converted_pred[ex_idx, bbox_idx, :]])
        all_bboxes.append(bboxes)

    return all_bboxes

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [24]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/input_dataset/kaggle.json ~/.kaggle/n
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [25]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("aladdinpersson/pascalvoc-yolo")

# print("Path to dataset files:", path)

In [26]:
class VOCDataset(torch.utils.data.Dataset):
  def __init__(
      self, csv_file, img_dir, label_dir, S=7, B=2, C=20, transform=None
  ):
    self.annotations = pd.read_csv(csv_file)
    self.img_dir = img_dir
    self.label_dir = label_dir
    self.transform = transform
    self.S = S
    self.B = B
    self.C = C

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
    label_path = os.path.join(self.label_dir, self.annotations.iloc[index, 1])
    boxes = []

    with open(label_path) as f:

      for label in f.readlines():
        class_label, x, y, width, height = [ float(x) if float(x) != int(float(x)) else int(x) for x in label.replace("\n", "").split() ]
        boxes.append([class_label, x , y, width, height])

    img_path = os.path.join(self.img_dir, self.annotations.iloc[index, 0])
    image = Image.open(img_path)
    boxes = torch.tensor(boxes)

    if self.transform:
      # image
      image, boxes = self.transform( image, boxes)

    label_matrix = torch.zeros((self.S, self.S, self.C + 5 * self.B))

    for box in boxes:
      class_label, x , y, width, height = box.tolist()
      class_label = int(class_label)
      i ,j = int(self.S*y), int(self.S*x)
      x_cell, y_cell  = self.S*x-j, self.S*y-i
      width_cell, height_cell = (width*self.S, height*self.S)

      if label_matrix[i, j, 20] == 0:
        label_matrix[i, j, 20] =1
        box_coordinates = torch.tensor(
            [x_cell, y_cell, width_cell, height_cell]
        )
        label_matrix[i, j, 21:25] = box_coordinates
        label_matrix[i, j, class_label] = 1

    return image, label_matrix


In [27]:
# Hyperparameters
LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE =16
WEIGHT_DECAY = 0
# computational reasons
EPOCHS = 100
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
LOAD_MODEL_FILE = "overfit.pth.tar"
IMG_DIR= "/content/images"
LABEL_DIR = "/content/labels"

class Compose(object):
  def __init__(self, transforms):
    self.transforms = transforms

  def __call__(self, img, bboxes):
    for t in self.transforms:
      img, bboxes = t(img), bboxes
    return img, bboxes

transform = Compose( [transforms.Resize((448, 448)), transforms.ToTensor()])

def train_fn(train_loader, model, optimizer, loss_fn):
  loop = tqdm(train_loader, leave = True)
  mean_loss = []

  for batch_idx, (x, y) in enumerate(loop):
    x, y  = x.to(DEVICE), y.to(DEVICE)
    out = model(x)
    loss = loss_fn(out, y)
    mean_loss.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # update progress bar
    loop.set_postfix(loss = loss.item())

  print(f"Mean loss was {sum(mean_loss)/len(mean_loss)}")

def main():
  model = YoloV1(split_size = 7, num_boxes = 2, num_classes = 20).to(DEVICE)
  optimizer = optim.Adam(
      model.parameters(), lr = LEARNING_RATE, weight_decay = WEIGHT_DECAY
  )
  loss_fn = TotalLoss()

  if LOAD_MODEL:
    load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

  train_dataset = VOCDataset(
      "/content/8examples.csv",
      transform = transform,
      img_dir = IMG_DIR,
      label_dir = LABEL_DIR
  )

  test_dataset = VOCDataset(
      "/content/test.csv",
      transform = transform,
      img_dir = IMG_DIR,
      label_dir = LABEL_DIR
  )

  train_loader = DataLoader(
      dataset = train_dataset,
      batch_size = BATCH_SIZE,
      num_workers = NUM_WORKERS,
      pin_memory = PIN_MEMORY,
      shuffle = True,
      drop_last = False
  )

  test_loader = DataLoader(
      dataset = test_dataset,
      batch_size = BATCH_SIZE,
      num_workers = NUM_WORKERS,
      pin_memory = PIN_MEMORY,
      shuffle = True,
      drop_last = True
  )

  for epoch in range(EPOCHS):


    pred_boxes, target_boxes = get_bboxes(
        train_loader, model, iou_threshold = 0.5, threshold = 0.4
        )

    mean_avg_prec = mean_average_precision(
        pred_boxes, target_boxes, iou_threshold = 0.5, box_format = "midpoint"
    )
    print(f"Train mAP: {mean_avg_prec}")

    train_fn( train_loader, model, optimizer, loss_fn)

    if mean_avg_prec >0.9 :
      checkpoint = {
          "state_dict": model.state_dict(),
          "optimizer": optimizer.state_dict()
      }
      save_checkpoint(checkpoint, filename = LOAD_MODEL_FILE)
      import time
      time.sleep(10)






In [ ]:
if __name__ == "__main__":
  main()

Train mAP: 0.0


  0%|          | 0/1 [00:00<?, ?it/s]

Dewesh
torch.Size([8, 1470])
torch.Size([8, 7, 7, 30])
